# OLS Regression - Simple Train and Test

## Required Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math
import os

## Notbook Settings

In [ ]:
# set working directory
os.chdir(".")

# make sure it is set right
print(os.getcwd())

# make sure plots display in notebook
%matplotlib inline

## Data Load

Weekly marketing spend (in thousand's) by channel and the corresponding product sales (in million's).

In [ ]:
# read in the advertising data set
ad_df = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv", index_col = [0])

# look at the top rows
ad_df.head()

## EDA

In [ ]:
# describe the dataset
ad_df.describe()

In [ ]:
# look at the data types
ad_df.info()

In [ ]:
sns.pairplot(ad_df)

In [ ]:
sns.distplot(ad_df["sales"])

In [ ]:
sns.heatmap(ad_df.drop(columns = "sales").corr(), annot=True, cmap="YlGnBu", square = True)

## Training - Model 1

In [ ]:
# specifying the regression model
ols_m1 = linear_model.LinearRegression()

from sklearn.model_selection import train_test_split

# feature set
X = ad_df.drop(columns = "sales")

# target
y = ad_df["sales"]

# creating training / testings datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
# fit the regression model
ols_m1.fit(X_train,y_train)

## Assumption Checking

Errors should be independently and identically normally distributed with a mean of 0 and a fixed variance.

### Residuals vs. Fitted Values

In [ ]:
# residuals versus fitted values
def plot_fit(fitted_values, residuals):
    plt.scatter(fitted_values, residuals)
    plt.axhline(y = 0, color = "r")
    plt.xlabel("Fitted Values")
    plt.ylabel("Residuals")

In [ ]:
residuals = ols_m1.predict(X_train) - y_train

fitted_values = ols_m1.predict(X_train)

In [ ]:
# call
plot_fit(fitted_values, residuals)

### Distribution of Residuals

In [ ]:
# plot of residuals
def plot_residuals(residuals):
    residuals.name = "Residuals"
    sns.distplot(residuals)

In [ ]:
# call
plot_residuals(residuals)

## Training - Model 2

### Remove High Influence Point?

In [ ]:
X_train = X_train[residuals != residuals.max()]

y_train = y_train[residuals != residuals.max()]

In [ ]:
# specifying the regression model
ols_m2 = linear_model.LinearRegression()

# fit the regression model
ols_m2.fit(X_train,y_train)

## Assumption Checking

In [ ]:
residuals = ols_m2.predict(X_train) - y_train

fitted_values = ols_m2.predict(X_train)

plot_fit(fitted_values, residuals)

In [ ]:
plot_residuals(residuals)

## Hypothesis Testing

In [ ]:
# use statsmodels
olssm_m2 = sm.OLS(y_train, X_train).fit()

# print summary
olssm_m2.summary()

## Training - Model 3

### Drop Newspaper

In [ ]:
X_train_clean = X_train.drop(columns = ["newspaper"])
X_test_clean = X_test.drop(columns = ["newspaper"])

In [ ]:
# specifying the regression model
ols_m3 = linear_model.LinearRegression()

# fit the regression model
ols_m3.fit(X_train_clean,y_train)

## Assumption Checking

In [ ]:
residuals = ols_m3.predict(X_train_clean) - y_train

fitted_values = ols_m3.predict(X_train_clean)

plot_fit(fitted_values, residuals)

In [ ]:
plot_residuals(residuals)

## Final Model Interpretation

### Feature Importance?

In [ ]:
for estimate in zip(X_train.columns, ols_m1.coef_):
     print("Fit 1 estimate: ", estimate)

for estimate in zip(X_train.columns, ols_m2.coef_):
     print("Fit 2 estimate: ", estimate)
        
for estimate in zip(X_train_clean.columns, ols_m3.coef_):
     print("Fit 3 estimate: ", estimate)

### Intercept?

In [ ]:
print(ols_m1.intercept_)
print(ols_m2.intercept_)
print(ols_m3.intercept_)

## Testing

In [ ]:
# predicted
predicted_m1 = ols_m1.predict(X_test)
predicted_m2 = ols_m2.predict(X_test)
predicted_m3 = ols_m3.predict(X_test_clean)

# actual
validate = pd.DataFrame(y_test)

validate.columns = ['actual']

validate['m1'] = predicted_m1
validate['m2'] = predicted_m2
validate['m3'] = predicted_m3

validate.head(10)

## Accuracy?

In [ ]:
# mean squared error
def mse(df, model):
    mse = np.sum((validate['actual'] - validate[model])**2) / len(validate)
    print("The " + model + " Mean Squared Error is " + str(mse))
    
models = ["m1", "m2", "m3"]

for model in models:
    mse(validate, model)

# sklearn mse
# mean_squared_error(predicted, y_test)

## Next Steps

Can you spot any ways to improve the model? Is the OLS linear model appropriate here? Why do you think the R-squared value actually gets worse in model 2? Hint: Check the assumptions!!!
What about p-values???

## References

Data sourced from An Introduction to Statistical Learning
with Applications in R
Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani.